<h2 style="color:blue;">Lab Assignment 4: Text Classification with Deep Learning</h2>
 

**AUTHOR NAME : BALBIR SINGH**

**ASU ID :1233870107** 

**FILE CREATION DATE  :2/20/2025**


In [26]:
pip install --upgrade tensorflow

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: tensorflow in c:\users\balbi\anaconda\lib\site-packages (2.18.0)




[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
# Code Cell 1  - Import all the necessary libraries and restaurant review data.
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import os
import requests
import zipfile
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout 
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
# Load the restaurant review data
file_path = "C:\\Users\\balbi\\Downloads\\restaurant_reviews_az.csv"
df = pd.read_csv(file_path)
df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,IVS7do_HBzroiCiymNdxDg,fdFgZQQYQJeEAshH4lxSfQ,sGy67CpJctjeCWClWqonjA,3,1,1,0,"OK, the hype about having Hatch chili in your ...",2020-01-27 22:59:06
1,QP2pSzSqpJTMWOCuUuyXkQ,JBLWSXBTKFvJYYiM-FnCOQ,3w7NRntdQ9h0KwDsksIt5Q,5,1,1,1,Pandemic pit stop to have an ice cream.... onl...,2020-04-19 05:33:16
2,oK0cGYStgDOusZKz9B1qug,2_9fKnXChUjC5xArfF8BLg,OMnPtRGmbY8qH_wIILfYKA,5,1,0,0,I was lucky enough to go to the soft opening a...,2020-02-29 19:43:44
3,E_ABvFCNVLbfOgRg3Pv1KQ,9MExTQ76GSKhxSWnTS901g,V9XlikTxq0My4gE8LULsjw,5,0,0,0,I've gone to claim Jumpers all over the US and...,2020-03-14 21:47:07
4,Rd222CrrnXkXukR2iWj69g,LPxuausjvDN88uPr-Q4cQA,CA5BOxKRDPGJgdUQ8OUOpw,4,1,0,0,"If you haven't been to Maynard's kitchen, it'...",2020-01-17 20:32:57


In [22]:
# Code Cell 2 (5%) - Remove 3-star reviews and create a Sentiment column.
df = df[df['stars'] != 3]  # Remove reviews with 3-star ratings
df['Sentiment'] = df['stars'].apply(lambda x: 1 if x >= 4 else 0)  # Assign sentiment labels
df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,Sentiment
1,QP2pSzSqpJTMWOCuUuyXkQ,JBLWSXBTKFvJYYiM-FnCOQ,3w7NRntdQ9h0KwDsksIt5Q,5,1,1,1,Pandemic pit stop to have an ice cream.... onl...,2020-04-19 05:33:16,1
2,oK0cGYStgDOusZKz9B1qug,2_9fKnXChUjC5xArfF8BLg,OMnPtRGmbY8qH_wIILfYKA,5,1,0,0,I was lucky enough to go to the soft opening a...,2020-02-29 19:43:44,1
3,E_ABvFCNVLbfOgRg3Pv1KQ,9MExTQ76GSKhxSWnTS901g,V9XlikTxq0My4gE8LULsjw,5,0,0,0,I've gone to claim Jumpers all over the US and...,2020-03-14 21:47:07,1
4,Rd222CrrnXkXukR2iWj69g,LPxuausjvDN88uPr-Q4cQA,CA5BOxKRDPGJgdUQ8OUOpw,4,1,0,0,"If you haven't been to Maynard's kitchen, it'...",2020-01-17 20:32:57,1
5,kx6O_lyLzUnA7Xip5wh2NA,YsINprB2G1DM8qG1hbrPUg,rViAhfKLKmwbhTKROM9m0w,1,0,0,0,I stay at the Main Hotel at the Casino from Ju...,2020-07-14 16:43:23,0


In [23]:
# Code Cell 3 (10%) - Data processing and splitting the dataset.
X = df['text']  # Assuming the review text column is named 'text'
y = df['Sentiment']

# Split the dataset into training (80%) and testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Display dataset size
X_train.shape, X_test.shape

((35274,), (8819,))

In [30]:
#Code Cell 4: Download the pre-trained GloVe word embeddings (glove.6B.100d.txt) and prepare the embedding matrix for this review dataset. 

# Define paths for GloVe
glove_dir = "C:\\Users\\balbi\\Downloads"
glove_zip_path = os.path.join(glove_dir, "glove.6B.zip")
glove_text_path = os.path.join(glove_dir, "glove.6B.100d.txt")

# Ensure the directory exists
os.makedirs(glove_dir, exist_ok=True)

# Download GloVe embeddings if not available
if not os.path.exists(glove_text_path):
    print("Downloading GloVe embeddings...")
    url = "http://nlp.stanford.edu/data/glove.6B.zip"

    response = requests.get(url, stream=True)
    with open(glove_zip_path, "wb") as f:
        for chunk in response.iter_content(chunk_size=1024):
            f.write(chunk)

    print("Download complete. Extracting files...")

    # Extract only the required file
    with zipfile.ZipFile(glove_zip_path, "r") as zip_ref:
        if "glove.6B.100d.txt" not in zip_ref.namelist():
            raise FileNotFoundError("glove.6B.100d.txt not found in the zip file.")
        zip_ref.extract("glove.6B.100d.txt", path=glove_dir)

    print("Extraction complete.")

# Load the GloVe embeddings
print("Loading GloVe embeddings...")
embeddings_index = {}
with open(glove_text_path, encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype="float32")
        embeddings_index[word] = coefs

print(f"Loaded {len(embeddings_index)} word vectors.")

# Tokenize the review text
MAX_NUM_WORDS = 20000  # Keep only the top 20k words
MAX_SEQUENCE_LENGTH = 100  # Max length of each review

if "text" not in df.columns:
    raise ValueError("Column 'text' not found in DataFrame. Ensure the dataset has a 'text' column.")

# Convert all reviews to string (to avoid issues with NaN)
df["text"] = df["text"].astype(str)

# Split data into training and testing sets (80%-20%)
X_train, X_test, y_train, y_test = train_test_split(df["text"], df["Sentiment"], test_size=0.2, random_state=42, stratify=df["Sentiment"])

# Tokenize the text
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')

word_index = tokenizer.word_index
print(f"Unique words in dataset: {len(word_index)}")

# Prepare embedding matrix
EMBEDDING_DIM = 100
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

print("Embedding matrix prepared successfully.")


Loading GloVe embeddings...
Loaded 400000 word vectors.
Unique words in dataset: 33305
Embedding matrix prepared successfully.


In [31]:
# Code Cell 5  -  Build a GRU model with pre-trained GloVe embedding and show model performance
# Define the GRU model
model = Sequential([
    Embedding(input_dim=num_words, output_dim=EMBEDDING_DIM, weights=[embedding_matrix], 
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),  # Use pre-trained GloVe embeddings
    GRU(128, return_sequences=True),  # First GRU layer with return_sequences=True
    Dropout(0.3),
    GRU(64),  # Second GRU layer
    Dropout(0.3),
    Dense(1, activation="sigmoid")  # Binary classification output (0 or 1 sentiment)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
history = model.fit(X_train_pad, y_train, epochs=5, batch_size=32, validation_data=(X_test_pad, y_test))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_pad, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")


C:\Users\balbi\anaconda\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 203s 172ms/step - accuracy: 0.8087 - loss: 0.4381 - val_accuracy: 0.9275 - val_loss: 0.1862
Epoch 2/5
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 201s 170ms/step - accuracy: 0.9327 - loss: 0.1789 - val_accuracy: 0.9353 - val_loss: 0.1597
Epoch 3/5
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 203s 171ms/step - accuracy: 0.9423 - loss: 0.1533 - val_accuracy: 0.9393 - val_loss: 0.1527
Epoch 4/5
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 190s 172ms/step - accuracy: 0.9530 - loss: 0.1264 - val_accuracy: 0.9494 - val_loss: 0.1395
Epoch 5/5
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 188s 171ms/step - accuracy: 0.9632 - loss: 0.0999 - val_accuracy: 0.9458 - val_loss: 0.1450
276/276 ━━━━━━━━━━━━━━━━━━━━ 17s 62ms/step - accuracy: 0.9450 - loss: 0.1483
Test Accuracy: 0.9458


In [32]:
# Code Cell 6 (10%) - Build an LSTM model with pre-trained GloVe embedding and show model performance 
# Define the LSTM model
model_lstm = Sequential([
    Embedding(input_dim=num_words, output_dim=EMBEDDING_DIM, weights=[embedding_matrix], 
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),  # Use pre-trained GloVe embeddings
    LSTM(128, return_sequences=True),  # First LSTM layer with return_sequences=True
    Dropout(0.3),
    LSTM(64),  # Second LSTM layer
    Dropout(0.3),
    Dense(1, activation="sigmoid")  # Binary classification output (0 or 1 sentiment)
])

# Compile the model
model_lstm.compile(optimizer=Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
history_lstm = model_lstm.fit(X_train_pad, y_train, epochs=5, batch_size=32, validation_data=(X_test_pad, y_test))

# Evaluate the model
test_loss, test_accuracy = model_lstm.evaluate(X_test_pad, y_test)
print(f"LSTM Model Test Accuracy: {test_accuracy:.4f}")


Epoch 1/5
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 207s 179ms/step - accuracy: 0.7481 - loss: 0.5275 - val_accuracy: 0.9145 - val_loss: 0.2209
Epoch 2/5
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 198s 179ms/step - accuracy: 0.9158 - loss: 0.2161 - val_accuracy: 0.9283 - val_loss: 0.1818
Epoch 3/5
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 198s 179ms/step - accuracy: 0.9315 - loss: 0.1748 - val_accuracy: 0.9353 - val_loss: 0.1661
Epoch 4/5
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 199s 177ms/step - accuracy: 0.9372 - loss: 0.1636 - val_accuracy: 0.9393 - val_loss: 0.1593
Epoch 5/5
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 108s 98ms/step - accuracy: 0.9408 - loss: 0.1534 - val_accuracy: 0.9355 - val_loss: 0.1630
276/276 ━━━━━━━━━━━━━━━━━━━━ 9s 33ms/step - accuracy: 0.9332 - loss: 0.1712
LSTM Model Test Accuracy: 0.9355


In [33]:
# Code Cell 7 - Build a GRU model with trainable GloVe embeddings and show model performance 
# Define the GRU model with trainable embeddings
model_gru_trainable = Sequential([
    Embedding(input_dim=num_words, output_dim=EMBEDDING_DIM, weights=[embedding_matrix], 
              input_length=MAX_SEQUENCE_LENGTH, trainable=True),  # Trainable GloVe embeddings
    GRU(128, return_sequences=True),  # First GRU layer with return_sequences=True
    Dropout(0.3),
    GRU(64),  # Second GRU layer
    Dropout(0.3),
    Dense(1, activation="sigmoid")  # Binary classification output (0 or 1 sentiment)
])

# Compile the model
model_gru_trainable.compile(optimizer=Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
history_gru_trainable = model_gru_trainable.fit(X_train_pad, y_train, epochs=5, batch_size=32, validation_data=(X_test_pad, y_test))

# Evaluate the model
test_loss, test_accuracy = model_gru_trainable.evaluate(X_test_pad, y_test)
print(f"GRU Model with Trainable GloVe Test Accuracy: {test_accuracy:.4f}")


Epoch 1/5
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 110s 93ms/step - accuracy: 0.7963 - loss: 0.4607 - val_accuracy: 0.9367 - val_loss: 0.1699
Epoch 2/5
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 117s 106ms/step - accuracy: 0.9532 - loss: 0.1252 - val_accuracy: 0.9508 - val_loss: 0.1349
Epoch 3/5
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 164s 149ms/step - accuracy: 0.9753 - loss: 0.0719 - val_accuracy: 0.9507 - val_loss: 0.1433
Epoch 4/5
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 1449s 1s/step - accuracy: 0.9880 - loss: 0.0417 - val_accuracy: 0.9475 - val_loss: 0.1761
Epoch 5/5
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 137s 110ms/step - accuracy: 0.9936 - loss: 0.0219 - val_accuracy: 0.9466 - val_loss: 0.2244
276/276 ━━━━━━━━━━━━━━━━━━━━ 11s 41ms/step - accuracy: 0.9429 - loss: 0.2383
GRU Model with Trainable GloVe Test Accuracy: 0.9466


In [34]:
# Code Cell 8  - Build an LSTM model with trainable GloVe embeddings and show model performance 
# Define the LSTM model with trainable embeddings
model_lstm_trainable = Sequential([
    Embedding(input_dim=num_words, output_dim=EMBEDDING_DIM, weights=[embedding_matrix], 
              input_length=MAX_SEQUENCE_LENGTH, trainable=True),  # Trainable GloVe embeddings
    LSTM(128, return_sequences=True),  # First LSTM layer with return_sequences=True
    Dropout(0.3),
    LSTM(64),  # Second LSTM layer
    Dropout(0.3),
    Dense(1, activation="sigmoid")  # Binary classification output (0 or 1 sentiment)
])

# Compile the model
model_lstm_trainable.compile(optimizer=Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
history_lstm_trainable = model_lstm_trainable.fit(X_train_pad, y_train, epochs=5, batch_size=32, validation_data=(X_test_pad, y_test))

# Evaluate the model
test_loss, test_accuracy = model_lstm_trainable.evaluate(X_test_pad, y_test)
print(f"LSTM Model with Trainable GloVe Test Accuracy: {test_accuracy:.4f}")


Epoch 1/5
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 155s 132ms/step - accuracy: 0.7490 - loss: 0.5136 - val_accuracy: 0.9216 - val_loss: 0.2064
Epoch 2/5
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 125s 113ms/step - accuracy: 0.9415 - loss: 0.1642 - val_accuracy: 0.9444 - val_loss: 0.1568
Epoch 3/5
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 1384s 1s/step - accuracy: 0.9682 - loss: 0.0884 - val_accuracy: 0.9429 - val_loss: 0.1406
Epoch 4/5
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 112s 101ms/step - accuracy: 0.9817 - loss: 0.0587 - val_accuracy: 0.9490 - val_loss: 0.1439
Epoch 5/5
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 138s 98ms/step - accuracy: 0.9892 - loss: 0.0384 - val_accuracy: 0.9468 - val_loss: 0.1880
276/276 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - accuracy: 0.9441 - loss: 0.1992
LSTM Model with Trainable GloVe Test Accuracy: 0.9468


In [24]:
# Code Cell 9 - Use your best model in your Lab assignment 2 and show the performance 
# Choose the best model based on previous test accuracies
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
X = df['text']  # Assuming the review text column is named 'text'
y = df['Sentiment']  # Assuming the target variable is 'Sentiment'

# Split the dataset into training (80%) and testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Display dataset size
print(f"Training samples: {len(X_train)}, Testing samples: {len(X_test)}")
# Step 1: Apply TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # Keep top 5000 most frequent words
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)  # Transform training data
X_test_tfidf = tfidf_vectorizer.transform(X_test)  # Transform testing data

# Step 2: Train the Best Model (SVM with TF-IDF)
svm_model_tfidf = SVC(kernel='linear')  # Define the SVM model
svm_model_tfidf.fit(X_train_tfidf, y_train)  # Train the model on training data

# Step 3: Predict on the Test Set
y_pred_svm_tfidf = svm_model_tfidf.predict(X_test_tfidf)  # Make predictions on test data

# Step 4: Evaluate the Model
test_accuracy = accuracy_score(y_test, y_pred_svm_tfidf)  # Calculate accuracy
print(f"SVM (TF-IDF) Test Accuracy: {test_accuracy:.4f}")

# Display Classification Report
print("SVM (TF-IDF) Performance:")
print(classification_report(y_test, y_pred_svm_tfidf))


Training samples: 35274, Testing samples: 8819
SVM (TF-IDF) Test Accuracy: 0.9575
SVM (TF-IDF) Performance:
              precision    recall  f1-score   support

           0       0.93      0.91      0.92      2463
           1       0.97      0.97      0.97      6356

    accuracy                           0.96      8819
   macro avg       0.95      0.94      0.95      8819
weighted avg       0.96      0.96      0.96      8819



**Text Cell 10: Compare and comment on your observations on the performance of different deep learning models and different embeddings (i.e.,  LSTM and GRU).**
In this lab, we compared the performance of different deep learning models (LSTM and GRU) using pre-trained GloVe embeddings in both frozen and trainable settings. Below are the key observations:

- **SVM with TF-IDF Achieved the Best Performance**

Accuracy: 95.75% (Higher than deep learning models).
F1-score: 97% (Class 1) → Consistently high precision & recall.
This suggests that traditional ML models with strong text feature extraction (TF-IDF) can outperform deep learning models on structured text datasets.

- **GRU and LSTM with Pre-trained GloVe Performed Slightly Worse**
Accuracy: ~94%
Possible reason: Pre-trained embeddings might not fully capture the dataset’s sentiment-specific language.

- **Trainable GloVe Embeddings Improved GRU and LSTM Models Slightly**
Accuracy: ~94.5% (GRU slightly better than LSTM)
Why? The ability to fine-tune word embeddings allowed the models to adapt better to sentiment classification.

- **Deep Learning Models Take More Training Time**
SVM (TF-IDF) trains faster than GRU/LSTM since it doesn’t rely on complex sequential computations.
GRU/LSTM are better suited for long-text dependencies, whereas TF-IDF captures word importance efficiently.

- **Conclusion: When to Use Which Model?**
Use SVM (TF-IDF) for fast & high-accuracy sentiment classification.
Use GRU/LSTM when text has complex dependencies (e.g., conversational AI, longer text reviews).
Use trainable embeddings (instead of pre-trained GloVe) for deep learning models if the dataset language differs significantly from general corpora. 

**If computational efficiency and performance are the priorities, GRU with Trainable GloVe embeddings is the best deep learning model for this task. However, given that SVM with TF-IDF outperformed both (~95.75% accuracy), it suggests that traditional machine learning models with strong text feature extraction can sometimes be more effective than deep learning models in structured sentiment analysis tasks.**

**Text Cell 11:Acknowledge if you have used any GenAI tools in this assignment and anyone you have worked with on this assignment.** 

**<span style="color:GREEN;">Acknowledgment</span>**
 I acknowledge that I have used ChatGPT as a GenAI tool to assist with this assignment. It helped in understanding deep learning concepts, specifically the differences between GRU and LSTM models, as well as pre-trained vs. trainable GloVe embeddings. Additionally, ChatGPT provided guidance on structuring Python code, debugging errors, and comparing model performances effectively. While I leveraged this tool for clarification and optimization, all implementations and final analyses were reviewed and understood by me. Furthermore, I confirm that I worked independently on this assignment and did not collaborate with any classmates or external individuals.e


In [2]:
# Code Cell 12:Render HTML output for this assignment and make sure the output of each cell is shown. 
# HTML rendering
!pip install jupyter
!pip install nbconvert
!jupyter nbconvert "  LA4_Singh_Balbir.ipynb" --to html


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
[NbConvertApp] Converting notebook   LA4_Singh_Balbir.ipynb to html
[NbConvertApp] Writing 362339 bytes to   LA4_Singh_Balbir.html
